# PD881
## settings
## raw data

In [ ]:
import pandas as pd

# 文件位置定义
file_col_def = './data/PD881_Column_Definition_20201014.xlsx'
# file_raw_data = './data/PD881_raw.xlsx'
# file_data = './data/PD881_data.xlsx'
file_raw_data = './data/PD876_raw.xlsx'
file_data = './data/PD876_data.xlsx'

COL_IDX = "NIDonly"

In [ ]:
# 列名信息
df_col_def = pd.read_excel(file_col_def)
# col_def = df_col_def[df_col_def['SPSS数据保留']==1]['英文描述'].to_list()
col_def = df_col_def[df_col_def['数据分析保留']==1]['英文描述'].to_list()
# col_def

In [ ]:
# 原数据初过滤列，生成初始数据
df_raw_data = pd.read_excel(file_raw_data, index_col=COL_IDX)
df = df_raw_data[col_def]
df.to_excel(file_data)
del df_raw_data

## preprocessing
### load file

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler


file_col_def = './data/PD881_Column_Definition_20201014.xlsx'
# file_data = './data/PD881_data.xlsx'
file_data = './data/PD876_data.xlsx'
COL_IDX = "NIDonly"

# load init data
print("0. Load file", file_data)
df = pd.read_excel(file_data, index_col=COL_IDX)

0. Load file ./data/PD876_data.xlsx


### EDA

In [ ]:
# 探索性分析
df.describe(include="all").to_excel('./temp/df_describe_876.xlsx')

### rename columns

In [ ]:
# df = df.rename(columns={
#     'ori_col_name': 'new_col_name',
# })

### special rules

In [2]:
# column familyhis
def familyhis_replace_rule(x):
    if x.values[0] == ",":
        return np.nan
    elif x.values[0] == ",无,":
        return "无"
    else:
        return "是"

df[["familyhis"]] = df[["familyhis"]].apply(familyhis_replace_rule, axis=1)

# df.loc[:, "familyhis"] = df[["familyhis"]].apply(familyhis_replace_rule, axis=1)[0]

In [ ]:
# column V0_DIAG_HY
# 五分类 -> 十分类, 不需要了
# df[["V0_DIAG_HY"]] = df[["V0_DIAG_HY"]].apply(lambda x: 2 * x)

### drop rows

In [3]:
# 删除以下数据样本，其中单个样本超过40%的列为空值
len_cols = len(df.columns)
null_value_threshold = int(len_cols * 0.4)

drop_row_list = df[df.isnull().sum(axis=1) >= null_value_threshold].index.to_list()

print("1. Drop values")
print("1-1. Drop", len(drop_row_list), "rows, export to ./temp/rows_dropped_876.xlsx")
pd.DataFrame(data=drop_row_list, columns=[COL_IDX]).to_excel("./temp/rows_dropped_876.xlsx")
df.drop(drop_row_list, axis=0, inplace=True)

1. Drop values
1-1. Drop 317 rows, export to ./temp/rows_dropped_876.xlsx


### drop columns

In [4]:
# 删除以下数据特征，其中所有样本该数据特征超过40%为空值
len_rows = len(df)
null_col_threshold = int(len_rows * 0.4)
# drop_col_list = df.columns[df.describe(include="all").loc["count", :] <= null_col_threshold].to_list()

drop_col_list = df.columns[df.isnull().sum(axis=0) >= null_col_threshold].to_list()

print("1-2. Drop", len(drop_col_list), "cols, export to ./temp/cols_dropped_876.xlsx")
pd.DataFrame(data=drop_col_list, columns=["DROPPED_COLUMNS"]).to_excel("./temp/cols_dropped_876.xlsx")
df.drop(drop_col_list, axis=1, inplace=True)

1-2. Drop 4 cols, export to ./temp/cols_dropped_876.xlsx


In [5]:
df.describe()

,INPUT_AGE,ageonset,HEIGHT,FIRST_DIAG_AGE,PMH_ANESTHETIC,V0_AGE,V0_COURSE_MONTH,V0_DIAG_HY,V0_PD_WEIGHT,V0_BP_SIT_SYS,...,V0_NMSStotal,V0_ESStotal,V0_GDStotal,V0_RBDtotal,V0_snifftotal,V0_lsnifftotal,V0_rsmelltotal,V0_UPDRSIII_OFFtotal,V0_PDQ39total,V0_PDQ39Q30
count,559.000000,559.000000,555.000000,559.000000,531.000000,559.000000,555.000000,556.000000,530.000000,373.000000,...,553.000000,547.000000,541.000000,535.000000,519.000000,5.060000e+02,5.020000e+02,521.000000,552.000000,550.000000
mean,49.531306,44.468694,166.410811,45.974955,0.433145,49.781753,62.854054,2.133993,63.610755,135.024129,...,10.735986,6.067642,11.408503,3.835514,5.764933,-4.238528e+03,-4.272341e+03,29.866355,33.996377,0.620000
std,12.784203,13.415152,7.541502,13.695462,0.907388,12.743216,61.803888,0.868122,11.824312,97.109056,...,6.637543,4.306162,7.399790,2.792957,2.489097,9.546751e+04,9.584710e+04,15.310701,24.520520,0.928775
min,14.000000,0.000000,145.000000,0.000000,0.000000,16.000000,0.000000,1.000000,35.500000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-2.147483e+06,-2.147483e+06,0.000000,0.000000,0.000000
25%,40.500000,36.000000,160.000000,38.000000,0.000000,41.000000,20.000000,2.000000,55.000000,116.000000,...,6.000000,3.000000,5.000000,2.000000,4.000000,4.000000e+00,4.000000e+00,18.916667,15.000000,0.000000
50%,49.000000,44.000000,167.000000,46.000000,0.000000,49.000000,40.000000,2.000000,63.000000,126.000000,...,10.000000,5.000000,11.000000,3.000000,6.000000,5.000000e+00,5.000000e+00,28.000000,28.000000,0.000000
75%,61.000000,54.000000,172.000000,55.000000,1.000000,61.000000,81.500000,3.000000,70.000000,137.000000,...,15.000000,8.000000,17.000000,5.000000,7.000000,7.000000e+00,7.000000e+00,39.000000,48.250000,1.000000
max,84.000000,77.000000,190.000000,83.000000,8.000000,84.000000,355.000000,5.000000,105.000000,1400.000000,...,30.000000,23.000000,29.000000,13.000000,12.000000,1.100000e+01,1.200000e+01,100.454545,127.000000,4.000000


### encoding

In [5]:
# 定类列名信息（离散）
df_col_def = pd.read_excel(file_col_def)
col_discrete_def = \
    df_col_def[(df_col_def["数据分析保留"]==1)&(df_col_def["数据类型"]=="定类")]["英文描述"].to_list()
# col_discrete_def

# pandas onehot编码
df = pd.get_dummies(df, columns=col_discrete_def)

In [ ]:
# 离散数据序数编码器（如使用onehot，不要执行ordinal编码）
ordinal_enc_dict = {}
for col_name in col_discrete_def:
    # Create Ordinal encoder for col
    ordinal_enc_dict[col_name] = OrdinalEncoder(dtype="int64")
    col = df[col_name]
    
    # Select non-null values of col
    col_not_null = col[col.notnull()]
    reshaped_vals = col_not_null.values.reshape(-1, 1)
    encoded_vals = ordinal_enc_dict[col_name].fit_transform(reshaped_vals)
    
    # Store the values to non-null values of the column in users
    df.loc[col.notnull(), col_name] = np.squeeze(encoded_vals)

# 离散数据编码映射
{key: val.categories_[0].tolist() for key, val in ordinal_enc_dict.items()}

# for val in ordinal_enc_dict.values():
#     print(val.categories_)

### imputer

In [6]:
# 均值补差
imputer = SimpleImputer(strategy='mean')
df[:] = imputer.fit_transform(df)

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
df[:] = scaler.fit_transform(df)

### check missing values

In [10]:
# check missing values
print("2. Check missing values")
print("2-1. Index - Missing values detected:", df.index.isna().any())

col_missing_values = df.columns[df.isna().any().values==True].tolist()
print("2-2. Columns - Missing values detected:", len(col_missing_values)>0)
print(col_missing_values)

2. Check missing values
2-1. Index - Missing values detected: False
2-2. Columns - Missing values detected: False
[]


### export final data

In [11]:
# debug
df.to_excel('./temp/df_final_876.xlsx')